In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical


# Set paths
data_dir = r"C:\Users\User\OneDrive\Documents\handwritten_multilang_cnn\data\DevanagariHandwrittenCharacterDataset\Train"
classes = sorted(os.listdir(data_dir))  # Get all class folders
num_classes = len(classes)
print("Total Classes:", num_classes)

# Load images
X = []
y = []

for idx, label in enumerate(classes):
    class_dir = os.path.join(data_dir, label)
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (32, 32))  # Standard size
        X.append(img)
        y.append(idx)

X = np.array(X).reshape(-1, 32, 32, 1).astype('float32') / 255.0
y = to_categorical(y, num_classes)

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32,32,1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Save best model
checkpoint = ModelCheckpoint("saved_models/devanagari_cnn_model.h5", save_best_only=True, monitor="val_accuracy", mode="max")

# Train
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64, callbacks=[checkpoint])


Total Classes: 46


KeyboardInterrupt: 

In [ ]:
# Plot accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.title('Accuracy over epochs')
plt.show()


In [ ]:
def predict_character(img_path, model, class_labels):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (32, 32))
    img = img.reshape(1, 32, 32, 1) / 255.0
    pred = model.predict(img)
    class_idx = np.argmax(pred)
    return class_labels[class_idx]

from tensorflow.keras.models import load_model
model = load_model("saved_models/devanagari_cnn_model.h5")
print(predict_character(r"C:\Users\User\Downloads\test.jpg", model, classes))

In [ ]:
import tensorflow as tf
print(tf.__version__)
